<a href="https://colab.research.google.com/github/anuragkadian/Topic-modelling/blob/main/LDA%20Applied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 9.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=fa2cc0f29d2fd2dd5d9aca7faaf8afe6edbee71b9d378bdb4480f54b5ed0879b
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [1]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import pandas as pd
import spacy

In [4]:
from spacy import displacy
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary
import sklearn
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [5]:
import warnings
import os
warnings.filterwarnings('ignore') 

In [8]:
df = pd.read_csv("Q37 OTs.csv")
df.head(5)

,Reviews
0,zero - i dont trade ever from a mobile...
1,You have done a great job of establishing a cl...
2,Yes its easy to use
3,Yes
4,Wide range of different stocks. App easy to use


In [9]:
d = list(df['Reviews'])

# joining with space content of text
text = '\n'.join(map(str, d))
 

#displaying result
print(text)

zero - i dont trade ever from a mobile...
You have done a great job of establishing a clean UI.     I would like to be able to set notifications for certain Watchlists/companies. The options here would need to be very specific, I.e. 'turn on 'announcements' notifications, for Watchlist A'
Yes its easy to use
Yes
Wide range of different stocks. App easy to use
Whilst I use desktop trading Algorithms for making trades, mobile platforms are essential for being able to monitor these trades
When placing a trade being able to see the chart on a mobile phone (have a small buy/sell pop up within the chart screen)
we need some features that trading view provide like easy charting and use indicators  and its smooth to use and easy to use not complex to use the tools like IG (Hard To find the tools)   we can choose the leverage on etoro 2* or *5  there is no commission for opening a trade like ig that said there is no commission on share trading but there was.  but I will use it as my feature bro

In [10]:
nlp = spacy.load('en_core_web_sm')

my_stop_words = [u'say', u'\'s', u'mr', u'be', u'said', u'says', u'saying', 'today', 'happy', 'nil', 'nan', u'well', 'well', 'ease', 'easy', 'plus', 'saxo', 'like', 'use', 'good', 'etc']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

doc = nlp(text)

doc

zero - i dont trade ever from a mobile...
You have done a great job of establishing a clean UI.     I would like to be able to set notifications for certain Watchlists/companies. The options here would need to be very specific, I.e. 'turn on 'announcements' notifications, for Watchlist A'
Yes its easy to use
Yes
Wide range of different stocks. App easy to use
Whilst I use desktop trading Algorithms for making trades, mobile platforms are essential for being able to monitor these trades
When placing a trade being able to see the chart on a mobile phone (have a small buy/sell pop up within the chart screen)
we need some features that trading view provide like easy charting and use indicators  and its smooth to use and easy to use not complex to use the tools like IG (Hard To find the tools)   we can choose the leverage on etoro 2* or *5  there is no commission for opening a trade like ig that said there is no commission on share trading but there was.  but I will use it as my feature bro

In [11]:
texts, article = [], []
for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and w.text != 'I':
        # we add the lematized version of the word
        article.append(w.lemma_)
    # if it's a new line, it means we're onto our next document
    if w.text == '\n':
        texts.append(article)
        article = []

texts

bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]
texts[10]

['satisfied', 'current', 'feature']

In [12]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# TFIDF = models.TfidfModel(corpus) # Fit TF-IDF model
# trans_TFIDF = TFIDF[corpus] # Apply TF-IDF model

ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)
ldamodel.show_topics()

[(0,
  '0.019*"trade" + 0.017*"trading" + 0.016*"mobile_app" + 0.014*" " + 0.014*"market" + 0.013*"platform" + 0.011*"  " + 0.011*"indicator" + 0.009*"option" + 0.009*"chart"'),
 (1,
  '0.022*"trade" + 0.017*"app" + 0.016*" " + 0.015*"  " + 0.014*"mobile_platform" + 0.011*"chart" + 0.010*"well" + 0.009*"order" + 0.009*"feature" + 0.007*"great"'),
 (2,
  '0.038*" " + 0.015*"trade" + 0.014*"trading" + 0.012*"app" + 0.011*"account" + 0.011*"easy" + 0.010*"stock" + 0.009*"time" + 0.008*"fund" + 0.008*"well"'),
 (3,
  '0.023*" " + 0.014*"phone" + 0.013*"trade" + 0.012*"mobile" + 0.012*"trading" + 0.010*"market" + 0.010*"app" + 0.010*"price" + 0.009*"indicator" + 0.009*"time"'),
 (4,
  '0.056*"chart" + 0.041*"trade" + 0.013*"mobile" + 0.010*"able" + 0.010*"app" + 0.010*"set" + 0.009*"access" + 0.007*"position" + 0.007*"datum" + 0.007*"place"')]

In [13]:
lsimodel = LsiModel(corpus=corpus, num_topics=5, id2word=dictionary)
lsimodel.show_topics()

[(0,
  '0.700*" " + 0.323*"trade" + 0.279*"app" + 0.154*"time" + 0.129*"trading" + 0.111*"  " + 0.097*"chart" + 0.096*"need" + 0.091*"stock" + 0.091*"market"'),
 (1,
  '-0.488*"trade" + 0.435*" " + -0.333*"chart" + -0.190*"trading" + -0.165*"mobile" + -0.140*"time" + -0.139*"market" + 0.109*"transfer" + -0.105*"feature" + -0.096*"able"'),
 (2,
  '-0.448*"trade" + 0.319*"chart" + -0.250*"add" + 0.242*"app" + -0.212*"fund" + 0.209*"market" + -0.175*"CopyTrader" + 0.157*"notification" + 0.147*"time" + -0.144*"portfolio"'),
 (3,
  '0.527*"chart" + -0.265*"market" + -0.250*"time" + -0.208*"dividend" + -0.182*"add" + -0.165*"hope" + -0.161*"notification" + 0.148*"trade" + -0.138*"fund" + -0.137*"CopyTrader"'),
 (4,
  '0.446*"chart" + -0.351*"trade" + 0.347*"add" + 0.263*"fund" + 0.255*"CopyTrader" + 0.192*"copy" + 0.161*"option" + -0.160*"app" + 0.128*"percentage" + 0.127*"money"')]

In [14]:
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.061333  0.021762       1        1  25.399273
4      0.087075  0.042469       2        1  21.259419
3     -0.063818 -0.021278       3        1  19.874117
0      0.037646 -0.099490       4        1  17.415566
1      0.000430  0.056537       5        1  16.051625, topic_info=           Term       Freq      Total Category  logprob  loglift
37        chart  55.000000  55.000000  Default  30.0000  30.0000
44               17.000000  17.000000  Default  29.0000  29.0000
43               62.000000  62.000000  Default  28.0000  28.0000
283  mobile_app  15.000000  15.000000  Default  27.0000  27.0000
2         trade  67.000000  67.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
403       think   2.212337   9.652535   Topic5  -5.4183   0.3562
0        mobile   2.735928  25.942605   Topic5  -5.2059  -0.4201
18       option   2.227431  14.062049   Topic5  -5.4115  -0.0133
35      trading   2.498043  34.875123   Topic5  -5.2969  -0.8069
38        phone   2.249712  17.542904   Topic5  -5.4016  -0.2245

[353 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
43        1  0.479986       
43        2  0.047999       
43        3  0.223993       
43        4  0.127996       
43        5  0.127996       
...     ...       ...    ...
839       2  0.660081  wrong
22        1  0.853329    yes
568       3  0.177830   zoom
568       4  0.533489   zoom
568       5  0.355659   zoom

[559 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 1, 2])